In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copernicusmarine as cm
import gc

/home/kshitiz/anaconda3/envs/new_atlantis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cm.login(username="kparihar", password="axv1xrv@nhu3WJW0jrh")

File /home/kshitiz/.copernicusmarine/.copernicusmarine-credentials already exists, overwrite it ? [y/N]:

INFO - 2024-07-31T21:14:29Z - Credentials file stored in /home/kshitiz/.copernicusmarine/.copernicusmarine-credentials.


True

Dataset: Global Ocean Biochemistry Hindcast (https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_BGC_001_029/description)
- dataset_id: "cmems_mod_glo_bgc_my_0.25deg_P1M-m",
- variables: ["fe","no3","o2","nppv","po4","si"],
- Parameter definitions:
    - fe [mmol/m3]: Mole concentration of dissolved iron in sea water
    - no3 [mmol/m3]: Mole concentration of nitrate in sea water
    - o2 [mmol/m3]: Mole concentration of dissolved molecular oxygen in sea water
    - nppv [mg/m3/day]: Net primary production of biomass expressed as carbon per unit volume in sea water
    - po4 [mmol/m3]: Mole concentration of phosphate in sea water
    - si [mmol/m3] :Mole concentration of silicate in sea water
- Spatial resolution: 0.25deg x 0.25deg
- It used PISCES biogeochemical model to perform simulation 

In [7]:
# Set parameters
data_request = {
    "dataset_id" : "cmems_mod_glo_bgc_my_0.25deg_P1M-m",
    "variables" : ["fe","no3","o2","nppv","po4","si"],
    "longitude" : [-180, 180], 
    "latitude" : [-90, 90],
    "time" : ["2021-03-01", "2021-03-31"],
    "depth": [None, None] 
}

In [8]:
# PISCES data
pisces = cm.read_dataframe(
    dataset_id=data_request["dataset_id"],
    variables=data_request["variables"],
    minimum_longitude=data_request["longitude"][0],
    maximum_longitude=data_request["longitude"][1],
    minimum_latitude=data_request["latitude"][0],
    maximum_latitude=data_request["latitude"][1],
    minimum_depth=data_request["depth"][0],
    maximum_depth=data_request["depth"][1],
    start_datetime=data_request["time"][0],
    end_datetime=data_request["time"][0]
)

INFO - 2024-07-31T21:14:49Z - Dataset version was not specified, the latest one was selected: "202406"
INFO - 2024-07-31T21:14:49Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-07-31T21:15:02Z - Service was not specified, the default one was selected: "arco-geo-series"
WARNING - 2024-07-31T21:15:04Z - Some or all of your subset selection [-90, 90] for the latitude dimension  exceed the dataset coordinates [-80.0, 90.0]
WARNING - 2024-07-31T21:15:04Z - Some or all of your subset selection [-180.0, 180.0] for the longitude dimension  exceed the dataset coordinates [-180.0, 179.75]
/home/kshitiz/anaconda3/envs/new_atlantis/lib/python3.12/site-packages/copernicusmarine/download_functions/subset_xarray.py:477: UserWarning: Discarding nonzero nanoseconds in conversion.
  return Timestamp(date).to_pydatetime()


In [9]:
# drop na rows
pisces = pisces.dropna()

print(pisces.shape)
pisces.head()

(40386181, 6)


fe        no3  \
depth   latitude longitude time                                                 
0.50576 -76.75   -180.00   2021-02-28 23:59:54.145275904  0.000534  25.972515   
                 -179.75   2021-02-28 23:59:54.145275904  0.000485  25.454300   
                 -179.50   2021-02-28 23:59:54.145275904  0.000414  24.827707   
                 -179.25   2021-02-28 23:59:54.145275904  0.000350  24.569496   
                 -179.00   2021-02-28 23:59:54.145275904  0.000315  24.575478   

                                                                  o2  \
depth   latitude longitude time                                        
0.50576 -76.75   -180.00   2021-02-28 23:59:54.145275904  347.191956   
                 -179.75   2021-02-28 23:59:54.145275904  347.055359   
                 -179.50   2021-02-28 23:59:54.145275904  347.648285   
                 -179.25   2021-02-28 23:59:54.145275904  347.700195   
                 -179.00   2021-02-28 23:59:54.145275904  347.331207   

                                                              nppv       po4  \
depth   latitude longitude time                                                
0.50576 -76.75   -180.00   2021-02-28 23:59:54.145275904  5.482745  1.857986   
                 -179.75   2021-02-28 23:59:54.145275904  5.096589  1.831087   
                 -179.50   2021-02-28 23:59:54.145275904  4.506510  1.799336   
                 -179.25   2021-02-28 23:59:54.145275904  3.978682  1.788129   
                 -179.00   2021-02-28 23:59:54.145275904  3.638510  1.790622   

                                                                 si  
depth   latitude longitude time                                      
0.50576 -76.75   -180.00   2021-02-28 23:59:54.145275904  48.271786  
                 -179.75   2021-02-28 23:59:54.145275904  47.030518  
                 -179.50   2021-02-28 23:59:54.145275904  45.697769  
                 -179.25   2021-02-28 23:59:54.145275904  45.570431  
                 -179.00   2021-02-28 23:59:54.145275904  46.035557

In [10]:
# remove depth and time as index and drop the time column 
pisces = pisces.reset_index(level=["depth", "time"]).drop(columns=["time"])
pisces.head()

depth        fe        no3          o2      nppv  \
latitude longitude                                                       
-76.75   -180.00    0.50576  0.000534  25.972515  347.191956  5.482745   
         -179.75    0.50576  0.000485  25.454300  347.055359  5.096589   
         -179.50    0.50576  0.000414  24.827707  347.648285  4.506510   
         -179.25    0.50576  0.000350  24.569496  347.700195  3.978682   
         -179.00    0.50576  0.000315  24.575478  347.331207  3.638510   

                         po4         si  
latitude longitude                       
-76.75   -180.00    1.857986  48.271786  
         -179.75    1.831087  47.030518  
         -179.50    1.799336  45.697769  
         -179.25    1.788129  45.570431  
         -179.00    1.790622  46.035557

In [11]:
# average across the depth for each (latitude, longitude)
pisces_cg = pisces.reset_index().groupby(["latitude","longitude"]).mean()
pisces_cg = pisces_cg.drop(columns=["depth"])

In [13]:
print(pisces_cg.shape)
pisces_cg.head()

(682424, 6)


fe        no3          o2      nppv       po4  \
latitude longitude                                                        
-76.75   -180.00    0.000600  28.258854  320.430878  1.840106  1.991930   
         -179.75    0.000560  28.171642  317.793701  1.751562  1.988375   
         -179.50    0.000552  28.044382  316.281830  1.552969  1.983901   
         -179.25    0.000523  27.970261  316.130310  1.386932  1.980938   
         -179.00    0.000514  27.984995  315.866150  1.274744  1.982690   

                           si  
latitude longitude             
-76.75   -180.00    60.155632  
         -179.75    61.047058  
         -179.50    62.005905  
         -179.25    62.312954  
         -179.00    62.672798

In [14]:
pisces_cg.to_csv("pisces_depth_cg.csv")

In [15]:
del pisces
gc.collect()

0

In [2]:
# load other datasets to combine with pisces
chl_cg = pd.read_csv("chl_cg.csv").set_index(["latitude","longitude"])
sst_cg = pd.read_csv("sst_cg.csv").set_index(["latitude","longitude"])
sssd_cg = pd.read_csv("sssd_cg.csv").set_index(["latitude","longitude"])
carbon_cg = pd.read_csv("carbon_cg.csv").set_index(["latitude","longitude"])
bathy_cg = pd.read_csv("bathy_cg.csv").set_index(["latitude","longitude"])
pisces_cg = pd.read_csv("pisces_depth_cg.csv").set_index(["latitude","longitude"])

In [3]:
all = chl_cg.join(sst_cg, how="inner").join(sssd_cg, how="inner").join(carbon_cg, how="inner").join(bathy_cg, how="inner").join(pisces_cg, how="inner")
all.head()

CHL  analysed_sst        sos        dos     fgco2  \
latitude longitude                                                           
-75.5    -180.00    0.745324    271.756394  34.204094  1027.5248  1.587086   
         -179.75    0.685044    271.701994  34.193108  1027.5159  1.428289   
         -179.50    0.621697    271.648394  34.180702  1027.5059  1.288172   
         -179.25    0.579071    271.599594  34.173040  1027.4995  1.154647   
         -179.00    0.686786    271.568394  34.167526  1027.4952  1.018365   

                    omega_ar  omega_ca        ph      spco2       talk  \
latitude longitude                                                       
-75.5    -180.00    1.466226  2.338735  8.087303  351.45520  2321.0903   
         -179.75    1.458978  2.327192  8.085031  353.45938  2320.8772   
         -179.50    1.454097  2.319419  8.083524  354.78280  2320.6514   
         -179.25    1.447437  2.308806  8.081387  356.69434  2320.5276   
         -179.00    1.438670  2.294847  8.078671  359.11487  2320.2573   

                         tco2     deptho        fe        no3         o2  \
latitude longitude                                                         
-75.5    -180.00    2194.7764  568.80536  0.000524  27.994402  326.66733   
         -179.75    2195.4310  573.67865  0.000505  27.846527  327.47052   
         -179.50    2195.7910  566.56550  0.000484  27.800156  327.06604   
         -179.25    2196.4705  565.98180  0.000495  27.945148  325.02423   
         -179.00    2197.2540  545.73870  0.000464  28.136585  322.88077   

                        nppv       po4         si  
latitude longitude                                 
-75.5    -180.00    0.864997  2.000869  68.964560  
         -179.75    0.872001  1.992910  68.159180  
         -179.50    0.870575  1.990123  67.640710  
         -179.25    0.865370  1.997842  67.783585  
         -179.00    0.888784  2.004793  66.788376

In [4]:
all.shape

(569724, 18)

In [5]:
all.to_csv("obs_chl_pisces_plus_other_features_march_2021.csv")